In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import pickle
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
from scipy.fftpack import dct
import gc

In [ ]:
#Load features
X = np.load('../SavedFeatures/X_f0.npy')
y = np.load('../SavedFeatures/y_f0.npy')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Đổi trường dữ liệu 
X_train = X_train.reshape(X_train.shape[0], -1)  # to 2D
X_val = X_val.reshape(X_val.shape[0], -1)  # to 2D

del X,y
gc.collect

In [6]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_val_predictions = xgb_model.predict(X_val)

In [ ]:
accuracy = accuracy_score(y_val, xgb_val_predictions)

f1_class_0 = f1_score(y_val, xgb_val_predictions, pos_label=0)  
f1_class_1 = f1_score(y_val, xgb_val_predictions, pos_label=1)  
f1_score = (f1_class_0 + f1_class_1)/2

# Tính EER

y_val_probabilities1 = xgb_model.predict_proba(X_val)[:, 1]
fpr1, tpr1, thresholds1 = roc_curve(y_val, y_val_probabilities1)
frr1 = 1 - tpr1
eer_index1 = np.nanargmin(np.abs(fpr1 - frr1))
eer1 = fpr1[eer_index1] 
eer_threshold1 = thresholds1[eer_index1]

y_val_probabilities = xgb_model.predict_proba(X_val)[:, 0]
fpr, tpr, thresholds = roc_curve(y_val, y_val_probabilities)
frr = 1 - tpr
eer_index = np.nanargmin(np.abs(fpr - frr))
eer = fpr[eer_index]  
eer_threshold = thresholds[eer_index]




print("====================XGB - F0====================")
print(f"Accuracy: {accuracy:.5f}")
print(f"F1-Score  {f1_score:.5f}")
print("\nClassification Report:\n", classification_report(y_val, xgb_val_predictions))
print("EER of Bonafine: ---------")
print(f"Equal Error Rate (EER): {eer:.5f}")
print(f"EER Threshold: {eer_threshold:.5f}")
print("EER of Spoof: ---------")
print(f"Equal Error Rate (EER): {eer1:.5f}")
print(f"EER Threshold: {eer_threshold1:.5f}")

====================XGB - F0====================
Accuracy: 0.82820
F1-Score  0.81519

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.91      0.86      1904
           1       0.85      0.70      0.77      1280

    accuracy                           0.83      3184
   macro avg       0.83      0.81      0.82      3184
weighted avg       0.83      0.83      0.82      3184

EER of Bonafine: ---------
Equal Error Rate (EER): 0.79989
EER Threshold: 0.71188
EER of Spoof: ---------
Equal Error Rate (EER): 0.20011
EER Threshold: 0.28847


In [8]:
# Save model as Pickle file
with open("../SavedModel/F0_xgb_dfd.pkl", 'wb') as file:
    pickle.dump(xgb_model, file)

In [ ]:
del accuracy, f1_class_0, f1_class_1, f1_score
del y_val_probabilities1, fpr1, tpr1, thresholds1, frr1, eer_index1, eer1, eer_threshold1
del y_val_probabilities, fpr, tpr, thresholds, frr, eer_index, eer, eer_threshold
del xgb_val_predictions

gc.collect()